In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

In [10]:
import torch
import torch.nn as nn

class SimpleMLP(nn.Module):
    """
    A network for generating image sequences using only MLP layers.
    """
    def __init__(self, input_dim, hidden_dim, num_layers):
        super(SimpleMLP, self).__init__()
        self.hidden_dim = hidden_dim

        # MLP Encoder layers
        encoder_layers = []
        current_dim = input_dim
        for _ in range(num_layers):
            encoder_layers.append(nn.Linear(current_dim, hidden_dim))
            encoder_layers.append(nn.ReLU())
            current_dim = hidden_dim
        self.encoder = nn.Sequential(*encoder_layers)

        # MLP Decoder layers
        decoder_layers = []
        current_dim = hidden_dim * 3  
        for _ in range(num_layers):
            decoder_layers.append(nn.Linear(current_dim, hidden_dim))
            decoder_layers.append(nn.ReLU())
            current_dim = hidden_dim
        decoder_layers.append(nn.Linear(hidden_dim, input_dim))
        self.decoder = nn.Sequential(*decoder_layers)

    def forward(self, x):
        batch_size, seq_len,num_channels,input_dim1,input_dim2 = x.size()
        output_inner = []

        for t in range(seq_len):
            for c in range(num_channels):
                xt = x[:, t, c,:,:].reshape(batch_size,-1)
                encoded_xt = self.encoder(xt)
                output_inner.append(encoded_xt)

        h_combined = torch.cat(output_inner, dim=1)

        final_output = self.decoder(h_combined)
        final_output=final_output.reshape(batch_size,input_dim1,input_dim2)

        return final_output



In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, Subset

# Assuming ConvLSTM and other necessary imports are already defined

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Parameters
time_steps = 1
channels = 1
height = 119
width = 149
batch_size = 64
num_epochs = 20


num_samples = len(all_segments)
train_size = int(0.8 * num_samples)
val_size = num_samples - train_size

train_indices = list(range(train_size))
val_indices = list(range(train_size, num_samples))
dataset=TensorDataset(all_segments, all_labels)
train_dataset = Subset(dataset, train_indices)
val_dataset = Subset(dataset, val_indices)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

model = SimpleMLP(input_dim=149*119, hidden_dim=64, num_layers=3).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.00001)

# Training Loop
for epoch in range(num_epochs):
    model.train()

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        sequences, labels = data
        sequences = sequences.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()


        outputs= model(sequences)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f'Epoch {epoch + 1}, Loss: {running_loss / len(train_loader):.3f}')

    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        for data in val_loader:
            sequences, labels = data
            sequences = sequences.to(device)
            labels = labels.to(device)

            outputs = model(sequences)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

        print(f'Validation Loss: {val_loss / len(val_loader):.3f}')

print('Finished Training')
